1. N + [RJ, RA, RV] 명사 오른쪽에는 “조사 / 형용사 / 동사"가 올 수 있다. 

1. 동/형용사 중에는 L/R이 안되는 경우가 있다. 이 경우에는 사전으로 외워두는게 낫다
    - 간 = 가/동사 + ㄴ/어미

사전을 만들 tag 종류는 다음과 같다. 
    
    ## 품사 사전
    Noun
    Adjective / Verb (어절이 단어임)
    Josa
    Determiner
    Adverb
    Exclamation
    Eomi
    
    ## LR 사전
    RVerb / RAdjective (명사 왼쪽에 등장하여 명사랑 결합되는 경우들: 이해했다)
    Josa
    Eomi / LVerb/LAdjective L-R 구조는 out of vocabulary 용언에 대한 힌트가 된다.

In [14]:
with open('../data/processed/lr/tokentable_simple_lr.txt', encoding='utf-8') as f:
    for i, _ in enumerate(f):
        if i >= 30: break
        print(_.strip())

text	morphomes	pos	lr	count
그	그/MM	그/Determiner	그/Determiner	80726
수	수/NNB	수/Noun	수/Noun	67991
이	이/MM	이/Determiner	이/Determiner	45777
것이다.	것/NNB+이/VCP+다/EF+./SF	것/Noun+이/Adjective+다/Eomi	것/Noun+이다/Adjective	37808
있다.	있/VX+다/EF+./SF	있/Verb+다/Eomi	있/Verb+다/Eomi	30548
한	한/MM	한/Determiner	한/Determiner	30536
있는	있/VX+는/ETM	있/Verb+는/Eomi	있/Verb+는/Eomi	28946
있다.	있/VV+다/EF+./SF	있/Verb+다/Eomi	있/Verb+다/Eomi	26195
있는	있/VV+는/ETM	있/Verb+는/Eomi	있/Verb+는/Eomi	25045
것은	것/NNB+은/JX	것/Noun+은/Josa	것/Noun+은/Josa	24816
그러나	그러나/MAJ	그러나/Adverb	그러나/Adverb	24498
같은	같/VA+은/ETM	같/Adjective+은/Eomi	같/Adjective+은/Eomi	21197
나는	나/NP+는/JX	나/Noun+는/Josa	나/Noun+는/Josa	21182
대한	대하/VV+ᆫ/ETM	대하/Verb+ᆫ/Eomi	대한/Verb+/Eomi	19874
그런	그런/MM	그런/Determiner	그런/Determiner	18999
것이	것/NNB+이/JKS	것/Noun+이/Josa	것/Noun+이/Josa	18870
그리고	그리고/MAJ	그리고/Adverb	그리고/Adverb	18322
하는	하/VV+는/ETM	하/Verb+는/Eomi	하/Verb+는/Eomi	17952
더	더/MAG	더/Adverb	더/Adverb	17355
안	안/MAG	안/Adverb	안/Adverb	17330
것을	것/NNB+을/JKO	것/Noun+을/Josa	것/Noun+을/Josa	16829
우리	우리/NP	우

In [71]:
from collections import defaultdict

def is_korean(s):
    for c in s:
        if not (44032 <= ord(c) <= 55203):
            return False
    return True


dictionary_pos = defaultdict(lambda: defaultdict(lambda: 0))
dictionary_r = defaultdict(lambda: defaultdict(lambda: 0))
dictionary_lv = defaultdict(lambda: defaultdict(lambda: 0))
dictionary_stem = defaultdict(lambda: defaultdict(lambda: 0))

with open('../data/processed/lr/tokentable_simple_lr.txt', encoding='utf-8') as f:
    next(f) # Skip head
    for line in f:
        _, _, pos, lr, count = line.split('\t')
        count = int(count.strip())
        
        pos = [p.split('/') for p in pos.split('+')]
        lr =  [w.split('/') for w in lr.split('+')]
        (ltag, rtag) = (lr[0][1], lr[-1][1])
        
        for word, tag in pos:
            if not is_korean(word):
                continue
            if (tag == 'Adjective' or tag == 'Verb'):
                dictionary_stem[tag][word] += count
                continue
            dictionary_pos[tag][word] += count
        if (ltag == 'Adjective' or ltag == 'Verb'):
            word = ''.join([w for w,_ in lr])
            if is_korean(word):
                dictionary_pos[ltag][word] += count

        if len(lr) == 2:
            if lr[1][0] and is_korean(lr[1][0]):
                dictionary_r[rtag][lr[1][0]] += count
            if ltag == 'Adjective' or ltag == 'Verb' and is_korean(lr[0][0]):
                dictionary_lv[ltag][lr[0][0]] += count

if 'SN' in dictionary_pos:
    del dictionary_pos['SN']
if 'Determiner' in dictionary_r:    
    del dictionary_r['Determiner']
if 'Exclamation' in dictionary_r:
    del dictionary_r['Exclamation']

In [72]:
len(dictionary_pos), len(dictionary_r), len(dictionary_lv),  len(dictionary_stem)

(8, 5, 2, 2)

In [73]:
def print_dictionary(d, head=''):
    print(head)
    for tag, words in d.items():
        print('%s has %d words' % (tag, len(words)))
    print('\n')

print_dictionary(dictionary_pos, '# dictionary_pos')
print_dictionary(dictionary_r, '# dictionary_r')
print_dictionary(dictionary_lv, '# dictionary_lv')
print_dictionary(dictionary_stem, '# dictionary_stem')

# dictionary_pos
Determiner has 205 words
Eomi has 17701 words
Exclamation has 1193 words
Adjective has 25618 words
Adverb has 6039 words
Noun has 326123 words
Josa has 2764 words
Verb has 141177 words


# dictionary_r
Eomi has 17227 words
Adjective has 5397 words
Adverb has 81 words
Josa has 7475 words
Verb has 6921 words


# dictionary_lv
Adjective has 7524 words
Verb has 16840 words


# dictionary_stem
Adjective has 4222 words
Verb has 7409 words




In [74]:
import os

root = '../data/processed/tagger/dictionary/'
for folder in ['pos', 'r', 'lv', 'stem']:
    dictionary = locals()['dictionary_{}'.format(folder)]
    folder = root + folder
    if not os.path.exists(folder):
        os.makedirs(folder)
    for tag, worddict in dictionary.items():
        with open('{}/{}.txt'.format(folder, tag), 'w', encoding='utf-8') as f:
            for word, count in sorted(worddict.items(), key=lambda x:-x[1]):
                f.write('{}\t{}\n'.format(word, count))

print('done')

done
